In [4]:
from dualnum import *
import numpy as np

In [5]:
RGAS = 8.314
H = 6.62607015e-34
NAV = 6.02214076e23

class EquationOfState:
    def pressure(self, T, V, n):
        return -self.helmholtz_energy(T, derive1(V), n).first_derivative

    def entropy(self, T, V, n):
        return -self.helmholtz_energy(derive1(T), V, n).first_derivative

    def chemical_potential(self, T, V, n):
        return self.helmholtz_energy(T, V, derive1(n)).first_derivative

    def isochoric_heat_capacity(self, T, V, n):
        return -self.helmholtz_energy(derive2(T), V, n).second_derivative*T/sum(n)

class IdealGas(EquationOfState):
    def __init__(self, MW):
        self.MW = MW

    def helmholtz_energy(self, T, V, n):
        de_broglie_lambda = [H*NAV/np.sqrt(2*np.pi*MWi*RGAS*T) for MWi in self.MW]
        return RGAS*T*sum(ni*(np.log(ni*NAV/V*li**3)-1) for ni,li in zip(n, de_broglie_lambda))

In [6]:
def helmholtz_energy(t, v, n, mw):
    H = 6.62607015e-34
    NAV = 6.02214076e23
    RGAS = 8.314
    if isinstance(n, list):
        n = np.array(n)
    de_broglie = H * NAV / np.sqrt(2.0 * np.pi * mw * RGAS * t)
    partial_density = n * NAV / V
    return RGAS * t * np.sum(n * (np.log(partial_density * de_broglie**3) - 1))

In [7]:
ig = IdealGas([39.948e-3, 4e-3])
mw = np.array([39.948e-3, 4e-3])

In [8]:
T = 300
V = 20
n = np.array([3, 2])

In [22]:
ig.helmholtz_energy(T, V, derive2(n)).second_derivative

[[831.3999999999999, 0.0], [0.0, 1247.1]]

In [23]:
helmholtz_energy(T, V, derive2(n), mw).second_derivative

[[831.3999999999999, 0.0], [0.0, 1247.1]]

In [28]:
ig.helmholtz_energy(derive2(T), V, n).second_derivative

-0.20784999999999998

In [29]:
helmholtz_energy(derive2(T), V, n, mw).second_derivative

-0.20784999999999998

In [11]:
helmholtz_energy(derive2(T), V, n, np.array([39.948e-3, 4e-3]))

-268235.1858577597 + [-956.4722861925324]ε1 + [-0.20784999999999998]ε1²

In [12]:
ig.isochoric_heat_capacity(T, V, n),RGAS*1.5

(12.471, 12.471)

In [13]:
ig.chemical_potential(T, V, n)

[-54192.23064420561, -46593.74696257142]

[[831.3999999999999, 0.0], [0.0, 1247.1]]

In [15]:
ig.helmholtz_energy(T, V, derive2(n)).second_derivative

[[831.3999999999999, 0.0], [0.0, 1247.1]]

In [16]:
[Td, Vd], nd = derive2([T, V], n)
[AT, AV], An = helmholtz_energy(Td, Vd, nd, mw).first_derivative
print(Td, Vd, nd)
helmholtz_energy(Td, Vd, nd, mw).second_derivative

300 + [1, 0]ε1 + [0, 0]ε2 + [[0, 0], [0, 0]]ε1ε2 20 + [0, 1]ε1 + [0, 0]ε2 + [[0, 0], [0, 0]]ε1ε2 [3 + [0, 0]ε1 + [1, 0]ε2 + [[0, 0], [0, 0]]ε1ε2, 2 + [0, 0]ε1 + [0, 1]ε2 + [[0, 0], [0, 0]]ε1ε2]


[[-193.11176881401872, -167.7834898752381], [0.0, 0.0]]

In [17]:
[Td, Vd], nd = derive2([T, V], n)
[AT, AV], An = ig.helmholtz_energy(Td, Vd, nd).first_derivative
print(Td, Vd, nd)
ig.helmholtz_energy(Td, Vd, nd).second_derivative

300 + [1, 0]ε1 + [0, 0]ε2 + [[0, 0], [0, 0]]ε1ε2 20 + [0, 1]ε1 + [0, 0]ε2 + [[0, 0], [0, 0]]ε1ε2 [3 + [0, 0]ε1 + [1, 0]ε2 + [[0, 0], [0, 0]]ε1ε2, 2 + [0, 0]ε1 + [0, 1]ε2 + [[0, 0], [0, 0]]ε1ε2]


[[-193.11176881401872, -167.7834898752381],
 [-124.71000000000005, -124.71000000000002]]

In [18]:
Td.__class__

HyperDualVec64

In [19]:
ig.helmholtz_energy(*derive2(T, V), n)

-268235.1858577597 + [-956.4722861925324]ε1 + [-623.5500000000001]ε2 + [-2.078500000000001]ε1ε2

In [20]:
-ig.helmholtz_energy(T, derive3(V), n).third_derivative,2*sum(n)*RGAS*T/V**3

(3.1177500000000005, 3.11775)